In [40]:
import pandas as pd
import itertools
from gurobipy import *
production = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "production")
setup = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "setup")
demand = pd.read_excel("data.xlsx", header = 0, index_col=0, sheet_name = "demand")

# Get the machines and parts from the setup sheet in excel
parts = list(setup.columns)
machines = list(setup.index)

print(parts)
# this is the working time in a week
combine_list = []
for r in itertools.product(machines, parts):
    combine_list.append(r[0]+ "_" +r[1]) 
# combine_list
# create dictionary for objective coefficient
setup_cost = {}
for v in combine_list:
    first = v.split("_")[0]
    second = v.split("_")[1]
    setup_cost[v] = setup.loc[first, second]


# maximum regular working hours per week
regular_hours_machine = {}
for name in machines:
    regular_hours_machine[name] = 120

# maximum overtime hours per week
overtime_hours = {}
for name in machines:
    overtime_hours[name] = 48



# Demand of each part
constraint_left_p_eq = {}
# example {"Part1":{"Machine1_Part1_Production":24, ...}}
for p in parts:
    production_co = {}
    for v in combine_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if second ==p:
            production_co[v] = production.loc[first, second]
        else:
            production_co[v] = 0
    constraint_left_p_eq[p] = production_co
# constraint_left_p_eq

# setup time for each machine
setup_time_machine = {}
# example {"Machine1":{"Machine1_Part1_Production":8, ...}}
for m in machines:
    setup_dic = {}
    for v in combine_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if first == m:
            setup_dic[v] = setup.loc[first, second]
        else:
            setup_dic[v] = 0
    setup_time_machine[m] = setup_dic
    

# production time for each machine
production_time_machine = {}
# example {"Machine1":{"Machine1_Part1_Production":24, ...}}
for m in machines:
    production_dic = {}
    for v in combine_list:
        first = v.split("_")[0]
        second = v.split("_")[1]
        if first == m:
            production_dic[v] = 1
        else:
            production_dic[v] = 0
    production_time_machine[m] = production_dic
# production_time_machine


# In[8]:


def optimize_model(week):
    # set up constraints name
    # Get demand for each part from demand table for week
    constraint_right_eq = {}
    for name in parts:
        constraint_right_eq[name] = demand.loc[week, name]

    m=Model()

    overtime_production_levels = m.addVars(combine_list, name="overtime")
    setup_levels = m.addVars(combine_list, name="setup", vtype = GRB.BINARY)
    regular_production_levels = m.addVars(combine_list, name="regular")
    
    overtime_leq = m.addConstrs((quicksum(production_time_machine[m][vp]*overtime_production_levels[vp] for vp in combine_list)
                                           <= overtime_hours[m] for m in machines), name="overtime_leq");

    
    regular_leq = m.addConstrs((quicksum(production_time_machine[m][vp]*regular_production_levels[vp] for vp in combine_list) + quicksum(setup_time_machine[m][vs]*setup_levels[vs] for vs in combine_list) 
                                           <= regular_hours_machine[m] for m in machines), name="regular_leq");
    
    eq_constraints_set = m.addConstrs((quicksum(setup_levels[v]*constraint_left_p_eq[p][v]*regular_production_levels[v] for v in combine_list) + quicksum(setup_levels[v]*constraint_left_p_eq[p][v]*overtime_production_levels[v] for v in combine_list)
                                           == constraint_right_eq[p] for p in parts ), name="eq_constraints"); 
    
    obj = quicksum(overtime_production_levels[vp] for vp in combine_list)
    m.update()
    m.setObjective(obj, GRB.MINIMIZE)
    m.update()
    m.optimize()
    cost = 0
    regular_hours = 0
    for variable in m.getVars():
        if variable.x != 0:
            name = variable.varName
            n1 = name.split("[")[0]
            n2 = name.split("[")[-1][:-1]
            if n1 == "setup":
                value = setup_cost[n2]*variable.x
                cost += setup_cost[n2]*variable.x*100
                regular_hours += setup_cost[n2]*variable.x
            elif n1 == "regular":
                value = variable.x
                cost += 100*variable.x
                regular_hours += variable.x
            else:
                value = variable.x
                cost += 150*variable.x
            print('%s %g' % (variable.varName, value))  

    print('Obj: %g' % m.ObjVal)
    print('total cost1: %g' % cost)
    print("regular hours: %g" % regular_hours)
    m.reset()


optimize_model(1)


optimize_model(2)


optimize_model(3)


optimize_model(4)


optimize_model(5)


optimize_model(6)


optimize_model(7)


optimize_model(8)


optimize_model(9)


optimize_model(10)


# PLease uncomment if needed. But the solution for this week is infeasible and hence the code might break.
# optimize_model(11)


optimize_model(12)






['Part1', 'Part2', 'Part3', 'Part4', 'Part5']
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model fingerprint: 0xc34d78de
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 47 columns
Presolve time: 0.00s
Presolved: 68 rows, 46 columns, 173 nonzeros
Variable types: 37 continuous, 9 integer (9 binary)

Root relaxation: objective 1.120894e+02, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  112.08943    0    6     

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model fingerprint: 0x42615523
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 4e+03]
Presolve removed 0 rows and 47 columns
Presolve time: 0.00s
Presolved: 68 rows, 46 columns, 173 nonzeros
Variable types: 37 continuous, 9 integer (9 binary)

Root relaxation: objective 9.198051e+01, 18 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   91.98051    0    6          -   91.98051      -     -    0s
H    0   

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 10 rows, 75 columns and 60 nonzeros
Model fingerprint: 0xb675c8bc
Model has 5 quadratic constraints
Variable types: 50 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [3e+03, 5e+03]
Presolve removed 1 rows and 48 columns
Presolve time: 0.00s
Presolved: 67 rows, 45 columns, 170 nonzeros
Variable types: 36 continuous, 9 integer (9 binary)

Root relaxation: objective 1.267085e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   12.67085    0    6          -   12.67085      -     -    0s
H    0   